**RNASeq Basics**

RNASeq data processing can be broadly divided into two types: 1) Reference Genome Based (Most commonly used) 2) De Novo, where no reference genome is available (uncommon, mostly used on novel species). For this course, we will be soley focussing on reference based RNA-Seq data analysis. Reference genomes for many species is available through consortia such as NCBI, Ensembl, etc. "Genome builds" are periodically released as more regions and genes are annotated within a genome (eg: hg19, hg39 (https://www.gencodegenes.org/human/)). 

**RNASeq data processing**

After sequencing is completed, you have a collection of sequencing reads for each sample in the form of a *fastq file*. In a reference-based RNAseq experiment, each sequencing read from a fastq file is *mapped* against the reference genome. There are several programs/workflows available to map reads to a genome such as TopHat suite (https://ccb.jhu.edu/software/tophat/index.shtml), STAR aligner (https://github.com/alexdobin/STAR) and StringTie suite (https://ccb.jhu.edu/software/stringtie/). RNASeq read aligners are *splicing-aware*, meaning they are able to map reads that skip introns / exons due to splicing events occuring across RNA transcripts.

Cellranger internally uses STAR alinger to perform mapping of reads to the human genome.

**Common things to consider after read mapping is completed :** 

1) What fraction of total reads successfully aligned to the genome ?

    Low fraction usually indicates contamination within your sample?

2) What fraction uniquely mapped to the genome ? 

    If each read has been derived from RNA molecules, each read must correspond to a unique location within the genome. However, reads failing to map uniquely to the genome is indicative of repetitive regions in your library (eg: ribosomal RNA).

3) What fraction of reads can be assigned to known, annotated genes ?
    
    We expect a high fraction of reads to be successfully assigned to know genes.

**Gene Expression & Normalization**

Once every read has been sufficiently mapped to a corresponding location across the genome, we can quantify the number of reads mapping across each gene/transcript/exon. Since the number of reads generated across a gene can be directly corelated with the length of the gene and also with the sequencing depth, the read counts across a sample must be normalized. There are several ways to perform read count normalization: 1) RPKM/FPKM (Reads/Fragments Per Kilobase of transcript per Million mapped reads) 2) TPM (Transcripts Per Million mapped reads) 3) RPM (Reads Per Million mapped reads)

## Some important RNASeq data terminologies :
We will be using these terms throughout the course

1) Fastq: This is a standardized file format to store sequencing reads for each sample. 

2) Reference genome : A representation set of sequences assembled through previous studies that best represents the organization of genes within a genome

3) Annotation : Each gene within a reference genome is assigned coordinates; eg: chr1:10,000-12,000. Each gene is further *annotated* by defining the location of UTRs, exons and introns within the gene. Commonly used annotations are provided from Gencode (https://www.gencodegenes.org/human/) and UCSC (https://genome.ucsc.edu/cgi-bin/hgTables).

4) Alignment / Alignment Tool / Aligners : Alignment is the process of identifying the region within the genome from which a read was derived. This is done by matching the read sequence with the genome sequence to find a perfect match.

5) SAM / BAM files : SAM file if a standardized file format to store alignment records for each read (https://samtools.github.io/hts-specs/SAMv1.pdf). A BAM file is a binary or compressed version of SAM files. BAM files are not human readable, unlike SAM files.

6) Gene Expression : Broad term for the number of reads derived from a gene (could be a normalized count).

7) Differential Gene Expression (DGE) : A gene that a expression significantly higher/lower between two experimental conditions.


---

![Diagram of RNA-Sequencing Data Analysis Pipeline](img/day1_00_overview-figure.png)

<i><b>Diagram of RNA-Sequencing Data Analysis Pipeline</b></i>

---



### Background on the FASTQ File Format

FASTQ files are text-based files for storing a biological sequence along with its corresponding quality scores, and are the most common file format that a bioinformatician would recieve from a sequencing run for further analysis. Both the sequence letter and quality score are each encoded with a single ASCII character. These files usually contain four lines per sequence. 

---

![FASTQ File Format Description](img/day1_02_fastq-file-format.png)

---

- **Line 1** begins with a `@` character and is followed by a sequence identifier and an *optional* description (like a `FASTA` title line)
- **Line 2** contains the raw sequence letters 
- **Line 3** begins with a `+` character and is *optionally* followed by the same sequence identifier (and any description) again
- **Line 4** encodes the quality values for the sequence in Line 2, and *must* contain the same number of symbols as letters in the sequence

It's important to become familiar with FASTQ files and how they're organized as they're one of the most common elements that you'll see across different bioinformatics pipelines that involve sequencing data (regardless of sequencing method)! 

<div class="alert alert-block alert-info">
    <p>If you have <b>paired-end</b> sequencing data, you will usually get <i>two</i> FASTQ files from the sequencing core, one labeled with an <code>R1</code> in its filename and one labeled with an <code>R2</code> in its filename!</p>
    <p>For <b>single-end</b> runs, then you will typically get <i>one</i> FASTQ file to evaluate.</p>
</div>


### Alignment File Format (SAM / BAM)

---

![SAM Alignment Column Information](https://www.michaelchimenti.com/wp-content/uploads/2018/06/sequence_string_sam-768x381.png)


---

## SAM File Flags

1 template having multiple segments in sequencing

2 each segment properly aligned according to the aligner

**4 segment unmapped**

8 next segment in the template unmapped

**16 SEQ being reverse complemented**

32 SEQ of the next segment in the template being reverse complemented

64 the first segment in the template

128 the last segment in the template

256 secondary alignment

512 not passing filters, such as platform/vendor quality controls

1024 PCR or optical duplicate

2048 supplementary alignment

## CIGAR Strings

**M alignment match / mismatch**

**I insertion to the reference**

**D deletion from the reference**

**N skipped region from the reference**

#################################################################

S soft clipping (clipped sequences present in SEQ)

H hard clipping (clipped sequences NOT present in SEQ)

P padding (silent deletion from padded reference)

= sequence match

X sequence mismatch

## CIGAR strings

The CIGAR string is a sequence of of base lengths and the associated operation. 

They are used to indicate things like which bases align (either a match/mismatch) with the reference, are deleted from the reference, and are insertions that are not in the reference.

### Example reference and read

RefPos:     1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19

Reference:  C  C  A  T  A  C  T  G  A  A  C  T  G  A  C  T  A  A  C

Read: ACTAGAATGGCT

### Read Alignment

```
RefPos:     1  2  3  4  5  6  7     8  9 10 11 12 13 14 15 16 17 18 19

Reference:  C  C  A  T  A  C  T     G  A  A  C  T  G  A  C  T  A  A  C

Read:                   A  C  T  A  G  A  A     T  G  G  C  T
```

### CIGAR String

```
CIGAR: 3M1I3M1D5M

Starting from the beginning of the read:

3 Matches / Mismatches

1 Insertion in the Read

3 Matches / Mismatches

1 Deletion in the read

5 Matches / Mismatches
```


### TODO
```

What is the CIGAR string for the following alignment ?

0123456789

AAGTCTAGAA (ref) 

  GT TAG   (query)
```

```
What is the CIGAR string for the following alignment ?

0123456789

AAGTCTAGAA (ref) 

  GT TAGT   (query)
```